# Functional Programming
~25min

### Intro

* By FP, I mean the functional code-writing style rather than the functional programming paradigm
    * Difference: paradigm is about the way you structure your code, style is about the way you write concrete things
    * Note: FP as paradigm deserves a separate course

Important: differentiating between functional and imperative style is not important; what matters is to know the tools at your disposal \
my task today is to show you the alternative way of writing the same code



imperative style:

```python
for i in range(10):
    print(i)
```

functional style:

```python
map(print, range(10)) # to evaluate the expression, use `list`
```


imperative style:

```python
def sum_squared(a, b):
    sum_ = a + b
    squared = sum_ ** 2
    return squared
```

functional style:

```python
def sum_squared(a, b):
    return (a + b) ** 2
```


In Python, functions are first-class citizens:

* can be passed as arguments to other functions
* can be returned from other functions
* can be stored in data structures

In [ ]:
def square(x):
    return x * x

# Assign to variable
f = square
print(f(4))  # 16


A **higher-order function** is a function that takes other functions as arguments or returns a function as its result. \
In simple words, a function working with functions.

In [ ]:

# Pass as argument
def apply_twice(func, val):
    return func(func(val))

print(apply_twice(square, 2))  # 16


In [ ]:

# Return a function
def make_multiplier(n):
    return lambda x: x * n

double = make_multiplier(2)
print(double(5))  # 10

Key features of FP:

* Higher-order functions (functions are passed as arguments, returned from other functions, and composed)
* Pure functions (no side effects, no hidden state)
    * Side effect: any change to the state of the program (e.g. changing a variable, printing to the console, etc.)
    * Hidden state: any change to the state of the program that is not immediately visible
* Immutability (passed data is never changed)

In [ ]:
map(print, range(10))


### `lambda`

Lambdas are one-use functions

In [ ]:
def string_len(string):
    return len(string)

data = ['when', 'the', 'sun', 'shines']
sorted(data, key=string_len)

In [ ]:
sorted(data, key=lambda string: len(string))

In [ ]:
# even better
sorted(data, key=len)

### Recursion

Recursion is a function calling itself until a base case; good for naturally nested data/structures.

In [ ]:
# idiomatic example: factorial
def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n - 1)

factorial(5)

More practical example: dictionary traversal \
Problem: we want to check that specific key-value pair exists in the dictionary

In [ ]:
config = {
    "user": {
        "name": "Ada",
        "profile": {
            "contacts": [
                {"type": "email", "value": "ada@lovelace.ai"},
                {"type": "phone", "value": "123-456"}
            ]
        }
    }
}

In [ ]:
def find_pair(config, key, value):
    if isinstance(config, dict):
        if key in config and config[key] == value:  # bug? can we compare any 2 objects?
            return True
        for k, v in config.items():
            if isinstance(v, (dict, list)):
                if find_pair(v, key, value):
                    return True
    elif isinstance(config, list):
        for item in config:
            if isinstance(item, (dict, list)):
                if find_pair(item, key, value):
                    return True
    return False

find_pair(config, "name", "Ada"), find_pair(config, "type", "email")

Tip: Python’s recursion depth is limited; for deep structures use iterative approaches or `sys.setrecursionlimit`.

### `map`

`map(f, iterable)` transforms each element of `iterable` using `f` and returns an iterator.

In [ ]:
words = ["the", "quick", "brown", "fox"]
upper = []
for w in words:
    upper.append(w.upper())
upper

In [ ]:
words = ["the", "quick", "brown", "fox"]
list(map(str.upper, words))  # wrap in list to realize the iterator

In [ ]:
# Note: list comprehensions are often more idiomatic
[word.upper() for word in words]

### `filter`

`filter(pred, iterable)` keeps elements where `pred(element)` is `True`; returns an iterator.

In [ ]:
nums = [5, -2, 0, 9, -7]
nonneg = []
for n in nums:
    if n >= 0:
        nonneg.append(n)
nonneg

In [ ]:
nums = [5, -2, 0, 9, -7]
list(filter(lambda n: n >= 0, nums))

Note: you can use list comprehensions instead of `filter`

### `reduce`

`reduce(func, iterable, init)` folds an iterable into one value by repeatedly combining elements.

In [ ]:
nums = [1, 2, 3, 4]
prod = 1
for n in nums:
    prod *= n
prod

In [ ]:
from functools import reduce
from operator import mul

nums = [1, 2, 3, 4]
reduce(mul, nums, 1)

Reality check: prefer built-ins when they exist (`sum(nums)`, `math.prod(nums)`). Use `reduce` when the combine-step isn’t a standard op.

optional

### `partial`

`partial(func, *frozen_args, **frozen_kwargs)` pre-fills some arguments, returning a new function with fewer parameters.

In [ ]:
def to_base2(s):
    return int(s, 2)

binaries = ["101", "1110", "1001"]
list(map(to_base2, binaries))

In [ ]:
from functools import partial

bin_to_int = partial(int, base=2)
binaries = ["101", "1110", "1001"]
list(map(bin_to_int, binaries))

optional

### `all` and `any`

`all(iterable)` returns `True` if all elements of `iterable` are true (or if the iterable is empty). \
`any(iterable)` returns `True` if any element of `iterable` is true. If the iterable is empty, returns `False`.

In [ ]:
fields = {"name": "Ada", "email": "ada@lovelace.ai", "age": 36}
ok = True
for v in fields.values():
    if not v:
        ok = False
        break
ok  # are all fields filled?

In [ ]:
all(fields.values())

In [ ]:
nums = [0, 0, 3, 0]
has_pos = False
for n in nums:
    if n > 0:
        has_pos = True
        break
has_pos

In [ ]:
nums = [0, 0, 3, 0]
any(n > 0 for n in nums)  # True if there exists a positive number

# Databases

~45min

### Why databases?

**Problem Statement:**

Delivery App: we want to store the data about our products, clients, and orders.

Functionality we want:
* Total price of the order
* List of orders for a given client


Take a look at [our data](data.json)

In [ ]:
from typing import Any, Dict, List
import json

class JsonStore:
    """
    Minimal wrapper over our single JSON 'database'.
    Required attributes: products, orders.
    """
    def __init__(self, data: Dict[str, Any]) -> None:
        self.products: List[Dict[str, Any]] = data.get("products", [])
        self.orders: List[Dict[str, Any]] = data.get("orders", [])

    def total_cost(self, order_id: int) -> float:
        """Sum qty * price (using the embedded product price) for a given order."""
        order = next((o for o in self.orders if o["id"] == order_id), None)  # iterate over ALL orders
        if order is None:
            raise ValueError(f"Order {order_id} not found")
        total = 0.0
        for item in order["items"]:
            price = float(item["product"]["price"])
            qty = int(item["qty"])
            total += price * qty
        return round(total, 2)

    def orders_by_client(self, client_id: int) -> List[Dict[str, Any]]:
        """Return all orders that match the given customer_id."""
        return [o for o in self.orders if o["customer_id"] == client_id]

with open("data.json", "r") as f:
    data = json.load(f)
db = JsonStore(data)
print("Total cost for order 1001:", db.total_cost(1001))
print("Orders for customer 1 (Alice):", [o["id"] for o in db.orders_by_client(1)])


Problems of the solution above:
* Data duplication
* All data is loaded into memory
* Performance is bad for large datasets
* We need to take care of data consistency (e.g. no products with the same id)
* ...


In simple words, it isn't good solution (not scalable, not flexible, etc.)







### DB: Intro

* **Database**: An organized collection of data that can be efficiently stored, queried, and managed.
* **Schema**: The structure (tables, columns, types, and relationships) that defines how data is organized in a database.


<img src="schema.png" alt="schema" width="50%" height="50%">


Great websites for visualizing DBs: [mermaid](https://mermaid.live), [drawsql](https://drawsql.app)

##### Relationships

* **One-to-Many (1→N)**:

  * Example: **Customer → Orders**
  * One customer can place many orders, but each order belongs to exactly one customer.

* **Many-to-Many (M↔N)**:

  * Example: **Orders ↔ Products**
  * An order can contain many products, and a product can appear in many orders.
  * This is implemented via a linking table: **Order\_Items**.




##### Keys

* **Primary Key (PK)**

  * A column (or set of columns) that **uniquely identifies each row** in a table.
  * Examples: `Customer.id`, `Product.id`, `Order.id`, `(Order_Item.order_id, Order_Item.product_id)`
  * Use: prevents duplicate rows, ensures every row has a unique identity.
  * Note: usually created automatically by the database.

* **Foreign Key (FK)**

  * A column in one table that **references the primary key in another table**.
  * Examples:

    * `Order.customer_id` → `Customer.id`
    * `Order_Item.order_id` → `Order.id`
    * `Order_Item.product_id` → `Product.id`
  * Use: enforces **referential integrity**, ensuring relationships are valid (no order with a non-existent customer, no order item with a non-existent product).


#### Terminology

* **Table**: like a list of rows, all with the same structure
* **Row / Record**: one entry in the table
* **Column**: named field with a type
* **Type**: the kind of data, e.g. `INTEGER`, `TEXT`, `REAL`
* **Primary Key (PK)**: unique identifier for each row
* **Foreign Key (FK)**: column linking to another table’s PK
* **Index**: structure that makes lookups faster





#### Types

| Database Type | Example Value     | Python Type  | Notes                                    |
|---------------|------------------|--------------|------------------------------------------|
| INTEGER       | 42               | int          | Whole numbers (IDs, counts, etc.)        |
| REAL          | 3.14             | float        | Decimal numbers (prices, measures)       |
| TEXT          | "Alice"          | str          | Strings of any length                    |
| DATE / DATETIME | "2025-09-12"   | str / datetime.date | Often stored as timestamp; Python’s `datetime` is more convenient |
| BOOLEAN       | 1 / 0 (true/false)| bool        | Some DBs use INTEGER under the hood      |
| BLOB          | binary data      | bytes        | For raw binary (images, files, etc.)     |


### SQL Cheat Sheet

**What we’re working with (schema):**
- `customer(id INTEGER PK, name TEXT UNIQUE NOT NULL)`
- `orders(id INTEGER PK, customer_id INTEGER NOT NULL, created_at TEXT DEFAULT CURRENT_TIMESTAMP, total REAL CHECK(total>=0), FK customer_id → customer(id) ON DELETE CASCADE)`

> All SQL lives in `sql/` and is executed via a tiny helper class. Foreign keys are enabled per connection inside the helper.


#### Setup


In [ ]:
from db_runner import DB
db = DB("mini.db")

**Content**

1. Init DB / enable foreign keys — `sql/00_init.sql`  
2. Create tables — `sql/01_create_tables.sql`  
3. Seed minimal data — `sql/02_seed.sql`  
4. SELECT basics — `sql/03_select_basics.sql`  
5. JOIN the two tables — `sql/04_join.sql`  
6. UPDATE & DELETE (cascade) — `sql/05_update_delete.sql`  
7. Transactions (commit/rollback) — `sql/06_transaction.sql`  
8. Index for a common query — `sql/07_index.sql`  
9. Upsert with ON CONFLICT — `sql/08_upsert.sql`


#### 1) Init DB / enable foreign keys
One-liner PRAGMA to ensure FK enforcement; keeps data consistent.
- **SQL file:** [sql/00_init.sql](sql/00_init.sql)  
- **Python:**


In [ ]:
db.run_file("sql/00_init.sql")


#### 2) Create tables
Define `customer` and `orders` with a cascading foreign key.
- **SQL file:** [sql/01_create_tables.sql](sql/01_create_tables.sql)  
- **Python:**


In [ ]:
db.run_file("sql/01_create_tables.sql")


#### 3) Seed minimal data
Add two customers and two example orders.
- **SQL file:** [sql/02_seed.sql](sql/02_seed.sql)  
- **Python:**


In [ ]:
db.run_file("sql/02_seed.sql")


#### 4) SELECT basics
Read everything, then filter/sort/limit results.
- **SQL file:** [sql/03_select_basics.sql](sql/03_select_basics.sql)  
- **Python:**


In [ ]:
for row in db.query_file("sql/03_select_basics.sql"):
    print(row)


#### 5) JOIN the two tables
Show orders with customer names, sorted by timestamp.
- **SQL file:** [sql/04_join.sql](sql/04_join.sql)  
- **Python:**


In [ ]:
for row in db.query_file("sql/04_join.sql"):
    print(row)


#### 6) UPDATE & DELETE (cascade)
Edit an order total; delete a customer and watch related orders vanish.
- **SQL file:** [sql/05_update_delete.sql](sql/05_update_delete.sql)  
- **Python:**


In [ ]:
db.run_file("sql/05_update_delete.sql")


### Solution

Create the database

In [ ]:
import sqlite3

def create_db(db_path="./data.db"):
    conn = sqlite3.connect(db_path)
    conn.execute("PRAGMA foreign_keys = ON")  # enforce FK in SQLite
    cur = conn.cursor()
    cur.executescript("""
    -- drop tables if they exist
    DROP TABLE IF EXISTS order_items;
    DROP TABLE IF EXISTS orders;
    DROP TABLE IF EXISTS products;
    DROP TABLE IF EXISTS customers;

    -- create tables
    CREATE TABLE customers (
      id   INTEGER PRIMARY KEY,
      name TEXT NOT NULL
    );

    CREATE TABLE products (
      id    INTEGER PRIMARY KEY,
      name  TEXT NOT NULL,
      price REAL  NOT NULL CHECK (price >= 0)
    );

    CREATE TABLE orders (
      id           INTEGER PRIMARY KEY,
      customer_id  INTEGER NOT NULL,
      created_at   TEXT    NOT NULL,   -- ISO date
      FOREIGN KEY (customer_id) REFERENCES customers(id)
    );

    -- snapshot price at purchase to preserve history
    CREATE TABLE order_items (
      order_id    INTEGER NOT NULL,
      product_id  INTEGER NOT NULL,
      quantity    INTEGER NOT NULL CHECK (quantity > 0),
      price_each  REAL    NOT NULL CHECK (price_each >= 0),
      PRIMARY KEY (order_id, product_id),
      FOREIGN KEY (order_id)   REFERENCES orders(id)   ON DELETE CASCADE,
      FOREIGN KEY (product_id) REFERENCES products(id) ON DELETE RESTRICT
    );
    """)
    conn.commit()
    return conn


Seed the database

In [ ]:
def seed(conn):
    cur = conn.cursor()

    cur.executemany("INSERT INTO customers(id, name) VALUES (?, ?)", [
        (1, "Alice"),
        (2, "Bob"),
        (3, "Cara"),
    ])

    cur.executemany("INSERT INTO products(id, name, price) VALUES (?, ?, ?)", [
        (1, "Coffee", 4.50),
        (2, "Tea",    3.00),
        (3, "Cookie", 2.50),
    ])

    cur.executemany("INSERT INTO orders(id, customer_id, created_at) VALUES (?, ?, ?)", [
        (1001, 1, "2025-09-05"),
        (1002, 2, "2025-09-08"),
        (1003, 3, "2025-09-11"),
    ])

    # items are embedded in orders in JSON; here they’re separate rows
    cur.executemany(
        "INSERT INTO order_items(order_id, product_id, quantity, price_each) VALUES (?, ?, ?, ?)",
        [
            (1001, 1, 2, 4.50),  # Alice: 2x Coffee
            (1001, 3, 1, 2.50),  # Alice: 1x Cookie
            (1002, 2, 1, 3.00),  # Bob:   1x Tea
            (1002, 1, 1, 4.50),  # Bob:   1x Coffee
            (1003, 2, 2, 3.00),  # Cara:  2x Tea
            (1003, 3, 3, 2.50),  # Cara:  3x Cookie
        ],
    )
    conn.commit()


Functionality

In [ ]:
from typing import List, Tuple

def ids_view(conn, client_id: int) -> Tuple[List[int], List[int]]:
    """
    Returns (all_client_ids, order_ids_for_given_client)
    """
    cur = conn.cursor()
    all_clients = [row[0] for row in cur.execute("SELECT id FROM customers ORDER BY id")]
    order_ids = [row[0] for row in cur.execute(
        "SELECT id FROM orders WHERE customer_id = ? ORDER BY created_at, id",
        (client_id,)
    )]
    return all_clients, order_ids


def total_cost(conn, order_id: int) -> float:
    """
    Returns total price for the given order_id (sum of quantity * price_each).
    """
    cur = conn.cursor()
    (total,) = cur.execute(
        "SELECT COALESCE(SUM(quantity * price_each), 0.0) FROM order_items WHERE order_id = ?",
        (order_id,)
    ).fetchone()
    return round(float(total or 0.0), 2)


In [ ]:
conn = create_db()
seed(conn)

clients, alice_orders = ids_view(conn, client_id=1)
print("Client IDs:", clients)                # -> [1, 2, 3]
print("Alice’s order IDs:", alice_orders)    # -> [1001]

print("Total cost (order 1001):", total_cost(conn, 1001))  # -> 11.50
print("Total cost (order 1003):", total_cost(conn, 1003))  # -> 13.00


Do you see a problem here? \
Google about SQL-Builder and SQLAlchemy

**Why Databases?**

* Handle **relationships** automatically (customers ↔ orders ↔ products)
* Keep **integrity**: foreign keys, unique IDs, no invalid references
* Make **queries fast** (e.g. with indexes and a lot of other optimizations)
* Allows to work with **large data** (no need to load everything into memory)
* **Most importantly you don't need to reinvent the wheel.**

Note: It doesn't mean that you should use DB whenever you work with data. Sometimes DBs are overkill